In [14]:
"""순서 
1. 해당 OTT 아이콘 클릭
2. 맨 아래까지 스크롤
3. 포스터 클릭 (상세페이지 접속)
4. 데이터 수집 (원하는 데이터의 선택자 찾기)
5. 나와서 3~4번 작업 반복"""


import requests
import time
import csv
import pandas as pd  
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from pandas import DataFrame

In [28]:
#티빙
driver = webdriver.Chrome(executable_path= r'C:\Users\u7rye\Desktop\aiTrack\chromedriver_win32\chromedriver.exe')
url = "https://seo.kinolights.com/discover/explore"
driver.implicitly_wait(30)
driver.get(url)
time.sleep(2)

In [29]:
#1. 해당 OTT(티빙) 아이콘 클릭 OK
css_selector = "#commentArea > div.filter-channels > div.header-inner-channels > div > div > button:nth-child(6) > i"
elem = driver.find_element_by_css_selector(css_selector)
elem.click()

In [30]:
#2. 맨 아래까지 스크롤
#현재 스크롤 높이
now_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    
    time.sleep(2)

    new_height = driver.execute_script("return document.body.scrollHeight")

    if(new_height == now_height): #전의 높이와 같다면 최하단이라고 판단해서 종료
        break
    
    now_height = new_height

print("스크롤 끝")    

KeyboardInterrupt: 

In [27]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

#컨테이너 수집
content = soup.select('div.mainContent > div.movie-list-item-wrap > div.MovieItem')

count = 1
articles = pd.DataFrame({"tvormovie": [], "name":[],'gerne':[],'keyward':[],'synopsis':[], 'director':[], 'actor':[], 'poster':[]})


for c in content:
    if os.path.exists('왓챠_콘텐츠.csv'):
        df = pd.read_csv(r"왓챠_콘텐츠.csv", low_memory=False)
        if c.select_one("div.title").text in df["name"].values:
            #print("continue")
            continue
        
    #3. 포스터 클릭(새 페이지 접속)
    href_src = c.find("a")["href"]

    page_url = "https://seo.kinolights.com" + href_src
    
    driver.get(page_url)
    
    html_each = driver.page_source
    soup = BeautifulSoup(html_each, 'html.parser')
    
    #영화별 컨테이너
    page_container = soup.select("main#contents")
    person_container = soup.select("div.container-person > div.person")
    
    #데이터 수집
    if not page_container[0].select_one("span.tv-label"):
        tvormovie = "영화"
    else:
        tvormovie = page_container[0].select_one("span.tv-label").text
    
    name = page_container[0].select_one("h3.title-kr").text.strip()
    
    genre = page_container[0].select_one("span.metadata-item").text.strip()
    genre = genre[:-1]
    
    poster = page_container[0].select_one("div.poster>img")
    poster_url = poster.get('src')
    
    keyward = page_container[0].select_one("ul.metadata-list").text
    keyward = keyward.strip().replace("\n", "").replace("              ", " ")
    
    synopsis = page_container[0].select_one("p.synopsis").text
    if "줄거리" in synopsis:
        synopsis = synopsis[5:]
    
    actor = ""
    director = ""
    for p in person_container:
        if "감독" in p.select_one("div.character").text:
            if p.select_one("div.name").text not in actor:
                director += p.select_one("div.name").text
                director += " "
        else:
            if p.select_one("div.name").text not in director:
                actor += p.select_one("div.name").text
                actor += " "
                
    #데이터프레임에 저장
    new_data = pd.DataFrame({"tvormovie":[tvormovie], "name":[name],'genre':[genre],'keyward':[keyward],'synopsis':[synopsis], 'director':[director], 'actor':[actor], 'poster':[poster_url]})
    #print("새 데이터 만들기 성공")
    
    if not os.path.exists('왓챠_콘텐츠.csv'):    
        new_data.to_csv('왓챠_콘텐츠.csv', mode='w', encoding='utf-8', index = False)
        print("new!")
    else:
        print("add")
        new_data.to_csv('왓챠_콘텐츠.csv', mode='a', encoding='utf-8', index = False, header=False)
        
    
    print(count)
    count += 1
    
print("완료 !")
driver.quit()

KeyboardInterrupt: 